# <img style="float: left; padding-right: 50px, width 45px"  src="https://miro.medium.com/max/1280/1*tHXPVLP294z8dcZwkL3wHw.png" width="250" height="50" > <br/><br/>
## TRON Bypass Rate Model ETL (Daily, Site Level)
 
**Team: RST-A**<br/>
**Date**: Q1 2023<br/>

<hr style="height:2pt" >

In [1]:
# Install potentially missing packages in SageMaker

# import sys
# ! {sys.executable} -m pip install tensorflow
# ! {sys.executable} -m pip install shap
# ! {sys.executable} -m pip install keras
# ! {sys.executable} -m pip install graphviz
# ! {sys.executable} -m pip install pyathena
# ! {sys.executable} -m pip install snappy
# ! {sys.executable} -m pip install xgboost
# ! {sys.executable} -m pip install seaborn

In [2]:
import datetime
from datetime import timedelta

import matplotlib.pylab as plt
import seaborn as sns

import pandas as pd
import boto3
import os
import io
import numpy as np
import math
import time
import statsmodels.api as sm

import sys
import pyarrow
import snappy
import graphviz

from itertools import chain
from functools import reduce
from pyathena import connect
from os import path

import xgboost

In [3]:
# Establishing S3 Data Directory

data_dir = 's3://cotezach-prod/TRON_Bypass_Modelling/'

In [4]:
# Function to Directly Query Athena Tables

def athena_conn(query, s3_dir = 's3://aws-athena-query-results-038954691342-us-east-1/', region = 'us-east-1'):
    """Return a SQL query from Athena into a pandas dataframe."""
    
    # Define connection to Athena
    conn = connect(s3_staging_dir = s3_dir,
                   region_name = region)
    
    # Query Athena and return to df
    df = pd.read_sql(query, conn)
    
    return df

In [5]:
# Fetch from / Write to S3 Functions

def fetch_s3_from_file(path_to_file):
    
    client = boto3.client('s3')
    obj2 = client.get_object(Bucket = 'cotezach-prod', Key = path_to_file)
    df = pd.read_csv(obj2['Body'])
    return df


def write_results_s3(dataframe, sand_box, path):
    data_string = dataframe.to_csv()
    resource = boto3.resource('s3')
    object = resource.Object(sand_box, path) # Select folder and file name in your path in S3
    object.put(Body = data_string)

## General Site Information

In [6]:
# Querying Building Information

query = '''

-- Building Information

WITH fc_info as (
            SELECT
             fc.warehouse AS building
            ,fc.region as region
            ,fc.timezone AS time_zone
            ,fc.building_type
            ,SPLIT_PART(z.drive_type,'-',2) as drive_type
            FROM ar_metadata.warehouses_v3_prod_v fc
            LEFT JOIN ar_metadata.zones_v3_prod_v z ON fc.warehouse = z.warehouse
            )
            
SELECT DISTINCT
     fc.building as site
    ,fc.region
    ,CASE WHEN fc.building_type = 'Gen11' THEN 'GEN11'
          WHEN fc.building_type = 'Sub Same Day Prime' THEN 'SSD'
          ELSE fc.building_type END as building_type
    ,fc.drive_type
    ,b.first_pick_data as pick_start
    ,b.first_stow_data as stow_start
FROM fc_info fc
JOIN aggregated_metrics.building_launch_info_v b on fc.building = b.site
WHERE 1=1
    AND fc.building IS NOT NULL
    AND fc.drive_type IS NOT NULL
    AND b.first_pick_data IS NOT NULL
    AND fc.drive_type IN ('H','G')
    AND fc.region IN ('NA','EU','NRT') '''

fc_data = athena_conn(query)

In [7]:
fc = fc_data

# Sorting by kiva_system_key & Datetime
fc = fc.sort_values(['site','drive_type','building_type','pick_start'], ignore_index = True)

# Creating Launch Year
fc['pick_start'] = pd.to_datetime(fc['pick_start'])
fc['launch_year'] = pd.DatetimeIndex(fc['pick_start']).year

# Trimming DF
fc = fc[['site','region','building_type','drive_type','pick_start','launch_year']]

fc.head()

,site,region,building_type,drive_type,pick_start,launch_year
0,ABQ1,NA,GEN11,H,2021-10-25,2021
1,ACY1,NA,Sortable,H,2018-12-29,2018
2,AGS1,NA,GEN11,H,2021-08-24,2021
3,AGS2,NA,Quick Deploy,H,2020-10-09,2020
4,AKC1,NA,Sortable,H,2020-11-04,2020


## Stow Performance Data

In [8]:
# Querying Stow Rate/Takt Data

query = '''

-- Daily Stow Rate/Takt by Site

WITH fc_info AS (
     SELECT 
         fc.warehouse as building
        ,fc.region as region
        ,fc.timezone as time_zone
     FROM ar_metadata.warehouses_v3_prod_v fc
    ),
    
stow_metrics as (
    SELECT s.*
    FROM ar_fulfillment.stow_metrics_prod_v s 
    WHERE 1=1
          AND pt_ingestion_date BETWEEN DATE('2022-01-01') AND CURRENT_DATE
          -- AND region IN ('NA','EU','NRT')
          -- AND station_operating_mode LIKE ('%IDS%')
          AND units_stowed > 0
          AND logged_milliseconds > 0
   ),
   
tenure_sub as (

-- This query is set ~2 months earlier than the main one above to ensure veteran rates are 100% captured

WITH hrs_per_day as(
     select
       warehouse as building,
       pt_ingestion_date as pt_date,
       user_id as uid,
       sum(logged_milliseconds/3600000.0) logged_hrs
     from ar_fulfillment.stow_metrics_prod_v s
     where pt_ingestion_date BETWEEN DATE('2021-11-01') AND CURRENT_DATE
     -- AND region IN ('NA')
     group by 1,2,3
 
), cumulative_hrs_table as (
   select
     building,
     pt_date,
     uid,
     sum(logged_hrs) over (partition by uid order by pt_date) cumulative_logged_hrs
   from hrs_per_day
)
 
   SELECT
     building,
     pt_date,
     uid,
     CASE WHEN cumulative_logged_hrs >= 160 THEN 'veteran' ELSE 'non' END as tenure
   from cumulative_hrs_table )

-- MAIN QUERY

SELECT
    warehouse as site
   -- ,zone as floor
   ,DATE(from_unixtime(to_unixtime(s.start_time),fc_info.time_zone)) AS pt_fc_date
   ,SUM(s.units_stowed*1.0) AS ib_volume
   ,SUM(s.logged_milliseconds*1.0/1000.0/3600.0) as stow_hrs
   ,SUM(CASE WHEN tenure = 'veteran' THEN s.logged_milliseconds*1.0/1000.0/3600.0 ELSE 0 END) AS vet_hrs
   ,SUM(s.work_milliseconds*1.0/1000.0) as stow_work_secs
   ,SUM(s.stows_performed_count) as stows_performed
   ,SUM(CASE WHEN station_operating_mode LIKE '%IDS%' THEN s.work_milliseconds*1.0/1000.0 ELSE 0 END) AS IDS_work_secs
   ,SUM(CASE WHEN station_operating_mode LIKE '%IDS%' THEN s.stows_performed_count ELSE 0 END) AS IDS_stows_performed
   ,SUM(NULLIF(pod_faces_count,0)*1.0) as pod_visits
   ,SUM(s.units_stowed*1.0)/ SUM(NULLIF(pod_faces_count,0)*1.0) AS units_per_face
FROM stow_metrics s
JOIN fc_info ON s.warehouse = fc_info.building
RIGHT JOIN tenure_sub t ON s.user_id = t.uid
GROUP BY 1,2 '''

# stow_data = athena_conn(query)

# Pulling in the Data from s3
stow_data = fetch_s3_from_file('TRON_Bypass_Modelling/stow_metrics.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [9]:
# if pulling in multiple CSVs from prodna/eu/nrt data sources

# stow_na = fetch_s3_from_file('TRON_Bypass_Modelling/stow_metrics_prodna.csv')
# stow_eu = fetch_s3_from_file('TRON_Bypass_Modelling/stow_metrics_prodeu.csv')
# stow_nrt = fetch_s3_from_file('TRON_Bypass_Modelling/stow_metrics_prodnrt.csv')

# Concatenate the Three DFs together
# stow_data = pd.concat([stow_na, stow_eu, stow_nrt], ignore_index = True)

In [10]:
stow = stow_data

# Setting pt_fc_date as Datetime
stow['pt_fc_date'] = pd.to_datetime(stow['pt_fc_date'])

# Sorting by kiva_system_key & Datetime
stow = stow.sort_values(['site','pt_fc_date'], ignore_index = True)

# Calc Key KPIs (tenure is considered LC5+ associates, >160 Stow hours logged)
stow['pod_visits'] = stow['ib_volume'] / stow['units_per_face']
stow['tenure_per'] = stow['vet_hrs'] / stow['stow_hrs'] *100
stow['ids_takt'] = stow['IDS_work_secs'] / stow['IDS_stows_performed']

stow.head()

,site,pt_fc_date,ib_volume,stow_hrs,vet_hrs,stow_work_secs,stows_performed,IDS_work_secs,IDS_stows_performed,pod_visits,units_per_face,tenure_per,ids_takt
0,1-Jan,2022-07-24,91.0,2.705768,0.000000,1.602306e+04,91.0,1.602306e+04,91,63.000000,1.444444,0.000000,176.077538
1,1-Jan,2022-07-25,929513.0,7884.598364,4335.538396,3.460000e+07,929513.0,3.460000e+07,929513,125281.000005,7.419425,54.987435,37.223794
2,1-Jan,2022-07-26,2205647.0,18385.576740,9893.401619,6.670000e+07,2205647.0,6.670000e+07,2205647,402150.000003,5.484638,53.810668,30.240560
3,1-Jan,2022-07-27,2347255.0,18709.798650,9649.500594,6.550000e+07,2347255.0,6.550000e+07,2347255,491475.999958,4.775930,51.574583,27.904936
4,1-Jan,2022-07-28,666783.0,6950.693949,2555.934894,2.560000e+07,666783.0,2.560000e+07,666783,144057.999987,4.628573,36.772370,38.393300


In [3]:
stow.describe

NameError: name 'stow' is not defined

## IDS-Specific Data

In [11]:
# Querying IDS Metrics (TBR, ME DPMO, etc.)

query = '''
    
-- NACF & EUCF Query
 
select
    n.building,
    -- n.floor,
    n.pt_date as pt_fc_date,
    SUM(n.stow_resolutions)*1.0 as total_stows,
    SUM(tron_high_confidence_agreements)*1.0 as hc_agreements,
    SUM(tron_high_confidence_agreements)*1.0/nullif(SUM(tron_high_confidence_audits)*1.0,0) as ml_accuracy,
    SUM(tron_high_confidence_audits)*1.0 as hc_audits,
    SUM(high_confidence_count)*1.0 as hc_count,
    SUM(tron_low_confidence_agreements)*1.0 as lc_agreements,
    SUM(tron_low_confidence_audits)*1.0 as lc_audits,
    SUM(low_confidence_count)*1.0 as lc_count,
    SUM(multiple_events)*1.0 as multiple_events,
    SUM(negative_stow_hint_violations)*1.0 as negative_stow_hint_violations
from aggregated_metrics_nike.ids_5min_aggregated_metrics_v n
where n.pt_date BETWEEN DATE('2022-01-01') AND CURRENT_DATE
group by 1,2
 
UNION ALL
 
-- NRT Query
 
select
    n.building,
    -- n.floor,
    n.pt_date as pt_fc_date,
    SUM(n.stow_resolutions)*1.0 as total_stows,
    SUM(tron_high_confidence_agreements)*1.0 as hc_agreements,
    SUM(tron_high_confidence_agreements)*1.0/nullif(SUM(tron_high_confidence_audits)*1.0,0) as ml_accuracy,
    SUM(tron_high_confidence_audits)*1.0 as hc_audits,
    SUM(high_confidence_count)*1.0 as hc_count,
    SUM(tron_low_confidence_agreements)*1.0 as lc_agreements,
    SUM(tron_low_confidence_audits)*1.0 as lc_audits,
    SUM(low_confidence_count)*1.0 as lc_count,
    SUM(multiple_events)*1.0 as multiple_events,
    SUM(negative_stow_hint_violations)*1.0 as negative_stow_hint_violations
from aggregated_metrics_nike.ids_5min_aggregated_metrics_nrt_v n
where n.pt_date BETWEEN DATE('2022-01-01') AND CURRENT_DATE
group by 1,2 '''

# ids_data = athena_conn(query)

# Pulling in the Data from s3
ids_data = fetch_s3_from_file('TRON_Bypass_Modelling/ids_metrics.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [12]:
ids = ids_data

# Setting pt_fc_date as Datetime
ids['pt_fc_date'] = pd.to_datetime(ids['pt_fc_date'])

# Renaming Building to Site
ids = ids.rename(columns={"building": "site"})

# Sorting by kiva_system_key & Datetime
ids = ids.sort_values(['site','pt_fc_date'], ignore_index = True)

# Calc Key KPIs
ids['tron_bypass_rate'] = ids['hc_count'] / ids['total_stows']
ids['tron_ioa'] = ids['lc_agreements'] / ids['lc_audits']
ids['me_dpmo'] = ids['multiple_events'] / ids['total_stows'] *1000000
ids['ml_accuracy'] = ids['ML_Accuracy']

In [13]:
# Imputing Missing Values using Nike Program Standards
ids['ml_accuracy'].fillna(0.999, inplace = True)
ids['tron_ioa'].fillna(0.993, inplace = True)

# Recalculating Ids System Accuracy with Imputed Values
ids['ids_accuracy'] = (ids['tron_bypass_rate'] * ids['ml_accuracy'] + (1 - ids['tron_bypass_rate']) * 
                       ids['tron_ioa'] - ids['me_dpmo']/1000000) *100

In [14]:
# Scaling all IDS % KPIs up by 100 for Modelling

ids['tron_bypass_rate'] = ids['tron_bypass_rate'] *100
ids['tron_ioa'] = ids['tron_ioa'] *100
ids['ml_accuracy'] = ids['ml_accuracy'] *100
ids['me_pct'] = ids['multiple_events'] / ids['total_stows'] *100 # Creating a more stable ME % metric

ids.head()

,site,pt_fc_date,total_stows,hc_agreements,ML_Accuracy,hc_audits,hc_count,lc_agreements,lc_audits,lc_count,multiple_events,tron_bypass_rate,tron_ioa,me_dpmo,ml_accuracy,ids_accuracy,me_pct
0,1-Jan,2022-06-21,2,0,NaN,0,1,0,0,1,0,50.000000,99.3,0.0,99.9,99.600000,0.0
1,1-Jan,2022-06-27,14,0,NaN,0,2,1,1,12,0,14.285714,100.0,0.0,99.9,99.985714,0.0
2,1-Jan,2022-06-28,50,0,NaN,0,13,2,2,37,1,26.000000,100.0,20000.0,99.9,97.974000,2.0
3,1-Jan,2022-06-29,73,0,NaN,0,10,1,1,63,0,13.698630,100.0,0.0,99.9,99.986301,0.0
4,1-Jan,2022-07-11,6,0,NaN,0,1,0,0,5,0,16.666667,99.3,0.0,99.9,99.400000,0.0


## Site Fullness & ASIN Size Data

In [15]:
# Querying GCU & ACU

query = '''

-- Daily GCU & ACU by Site
 
WITH fc_info AS (
     SELECT 
         fc.warehouse as building
        ,fc.region as region
        ,fc.timezone as time_zone
     FROM ar_metadata.warehouses_v3_prod_v fc
    ),
 
dedup_check as (
SELECT 
     a.warehouse
     ,zone
     ,snapshot_time
     ,DATE(at_timezone(snapshot_time, fc_info.time_zone)) pt_fc_date
     ,AVG(bins) bins
     ,AVG(units) units
     ,AVG(gross_volume_mm_cubed) gross_volume_in_cubed
     ,AVG(inventory_volume_mm_cubed) inventory_volume_in_cubed
     ,AVG(target_utilization_percent) target_cubic_utilization
     FROM ar_fulfillment.inventory_metrics_prod_v a
     JOIN fc_info ON a.warehouse = fc_info.building     
     WHERE 1=1
     and pt_ingestion_date BETWEEN DATE('2022-01-01') AND CURRENT_DATE
     and bin_template <> 'PALLET-SINGLE'
     and gross_volume_mm_cubed > 0
     group by 1,2,3,4 
     ),
     
snapshot_level AS (
SELECT 
    warehouse
    ,zone
    ,snapshot_time
    ,pt_fc_date
    ,SUM(bins) bins
    ,SUM(units) units
    ,SUM(gross_volume_in_cubed) gross_volume
    ,SUM(inventory_volume_in_cubed) inventory_volume
    FROM dedup_check
    GROUP BY 1,2,3,4
    ),

capacity_metrics AS (
SELECT
     warehouse as site
    ,zone
    ,pt_fc_date
    ,FLOOR(AVG(bins)) bins
    ,FLOOR(AVG(units)) units
    ,AVG(gross_volume) gross_volume
    ,AVG(inventory_volume) inventory_volume
    FROM snapshot_level
    GROUP BY 1,2,3
    )
    
-- MAIN QUERY

SELECT 
     site
    -- ,floor
    ,pt_fc_date
    ,SUM(units) units
    ,SUM(bins) bins
    ,SUM(inventory_volume*1.0) AS inventory_volume
    ,SUM(gross_volume*1.0) AS gross_volume
    ,100.0*SUM(inventory_volume*1.0)/SUM(gross_volume*1.0) gcu
    ,1.0*SUM(inventory_volume*1.0)/(12*12*12)/SUM(units) acu
FROM capacity_metrics
GROUP BY 1,2 '''

# gcu_data = athena_conn(query)

# Pulling in the Data from s3
gcu_data = fetch_s3_from_file('TRON_Bypass_Modelling/gcu_metrics.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [16]:
# if pulling in multiple CSVs from prodna/eu/nrt data sources

# gcu_na = fetch_s3_from_file('TRON_Bypass_Modelling/gcu_metrics_prodna.csv')
# gcu_eu = fetch_s3_from_file('TRON_Bypass_Modelling/gcu_metrics_prodeu.csv')
# gcu_nrt = fetch_s3_from_file('TRON_Bypass_Modelling/gcu_metrics_prodnrt.csv')

# Concatenate the Three DFs together
# gcu_data = pd.concat([gcu_na, gcu_eu, gcu_nrt], ignore_index = True)

In [17]:
gcu = gcu_data

# Setting pt_fc_date as Datetime
gcu['pt_fc_date'] = pd.to_datetime(gcu['pt_fc_date'])

# Sorting by kiva_system_key & Datetime
gcu = gcu.sort_values(['site','pt_fc_date'], ignore_index = True)

# Converting ACU to Cubic Inches (from Cubic Feet)
gcu['acu'] = gcu['acu'] * (12*12*12)

# Calc Units per Bin
gcu['units_per_bin'] = gcu['units'] / gcu['bins']

gcu.head()

,site,pt_fc_date,units,bins,inventory_volume,gross_volume,gcu,acu,units_per_bin
0,ABQ1,2021-12-31,1495631.0,621114.0,2.118180e+08,7.270151e+08,29.135290,141.624484,2.407981
1,ABQ1,2022-01-01,1495863.0,621114.0,2.118493e+08,7.270151e+08,29.139595,141.623440,2.408355
2,ABQ1,2022-01-02,1498728.0,621114.0,2.123313e+08,7.270151e+08,29.205901,141.674353,2.412968
3,ABQ1,2022-01-03,1508610.0,621114.0,2.140954e+08,7.270151e+08,29.448547,141.915666,2.428878
4,ABQ1,2022-01-04,1520291.0,621114.0,2.162039e+08,7.270151e+08,29.738573,142.212202,2.447684


## Site Podgap Data

In [18]:
# Querying Podgap

query = '''

-- Stow Podgap
 
WITH fc_info AS (
     SELECT 
         fc.warehouse as building
        ,fc.region as region
        ,fc.timezone as time_zone
     FROM ar_metadata.warehouses_v3_prod_v fc
    ),
    
pod_gap as (
    SELECT s.*
    FROM ar_fulfillment.pod_gap_metrics_prod_v s 
    WHERE 1=1
          AND pt_ingestion_date BETWEEN DATE('2022-01-01') AND CURRENT_DATE
          AND station_operating_mode LIKE ('%Stow%')
          -- AND region IN ('NA')
   )
 
-- MAIN QUERY
 
SELECT
    warehouse as site
   -- ,zone as floor
   ,DATE(from_unixtime(to_unixtime(s.start_time),fc_info.time_zone)) AS pt_fc_date
   ,SUM(s.gap_time_milliseconds*1.0) AS stow_gap_ms
   ,SUM(s.gap_time_milliseconds + s.ramp_up_time_milliseconds + s.dwell_time_milliseconds + s.transition_time_milliseconds) AS stow_total_ms     
FROM pod_gap s
JOIN fc_info ON s.warehouse = fc_info.building
GROUP BY 1,2 '''

# pg_data = athena_conn(query)

# Pulling in the Data from s3
pg_data = fetch_s3_from_file('TRON_Bypass_Modelling/podgap_metrics.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


In [19]:
# if pulling in multiple CSVs from prodna/eu/nrt data sources

# pg_na = fetch_s3_from_file('TRON_Bypass_Modelling/podgap_metrics_prodna.csv')
# pg_eu = fetch_s3_from_file('TRON_Bypass_Modelling/podgap_metrics_prodeu.csv')
# pg_nrt = fetch_s3_from_file('TRON_Bypass_Modelling/podgap_metrics_prodnrt.csv')

# Concatenate the Three DFs together
# pg_data = pd.concat([pg_na, pg_eu, pg_nrt], ignore_index = True)

In [20]:
podgap = pg_data

# Setting pt_fc_date as Datetime
podgap['pt_fc_date'] = pd.to_datetime(podgap['pt_fc_date'])

# Sorting by kiva_system_key & Datetime
podgap = podgap.sort_values(['site','pt_fc_date'], ignore_index = True)

# Calculating Podgap Percentage
podgap['stowgap'] = podgap['stow_gap_ms'] / podgap['stow_total_ms'] *100

podgap.head()

,site,pt_fc_date,stow_gap_ms,stow_total_ms,stowgap
0,1-Jan,2022-06-09,0.0,716695.0,0.000000
1,1-Jan,2022-06-10,0.0,948065.0,0.000000
2,1-Jan,2022-06-15,38615.0,1000856.0,3.858197
3,1-Jan,2022-06-16,334811.0,4099128.0,8.167859
4,1-Jan,2022-06-17,98484.0,8157976.0,1.207211


## Merging All Data Together

In [21]:
# Main DF is going to be the IDS Dataframe
main_df = ids

# Merging in all other KPIs
kpi_dfs = [main_df, stow, podgap, gcu] 
main_df = reduce(lambda left, right: pd.merge(left, right, on = ['site','pt_fc_date'], how = 'outer'), kpi_dfs)

# Merging with FC_INFO (for Drive Type & Region)
main_df = pd.merge(main_df, fc, how = 'left', on = ['site'])

main_df.columns.unique()

Index(['site', 'pt_fc_date', 'total_stows', 'hc_agreements', 'ML_Accuracy',
       'hc_audits', 'hc_count', 'lc_agreements', 'lc_audits', 'lc_count',
       'multiple_events', 'tron_bypass_rate', 'tron_ioa', 'me_dpmo',
       'ml_accuracy', 'ids_accuracy', 'me_pct', 'ib_volume', 'stow_hrs',
       'vet_hrs', 'stow_work_secs', 'stows_performed', 'IDS_work_secs',
       'IDS_stows_performed', 'pod_visits', 'units_per_face', 'tenure_per',
       'ids_takt', 'stow_gap_ms', 'stow_total_ms', 'stowgap', 'units', 'bins',
       'inventory_volume', 'gross_volume', 'gcu', 'acu', 'units_per_bin',
       'region', 'building_type', 'drive_type', 'pick_start', 'launch_year'],
      dtype='object')

In [22]:
# Trimming Main Df to just applicable Sites, Drive Types
main_df = main_df[main_df['building_type'].isin(['GEN11','Sortable','SSD','Quick Deploy'])]
main_df = main_df[main_df['drive_type'].isin(['H','G'])]

# Creating Maturity Binary Field (120 Days Since First Pick)
main_df['site_maturity'] = ((main_df['pt_fc_date'] - main_df['pick_start']).dt.days >= 120)*1

# What Buildings are Present?
main_df.site.unique()

array(['ABQ1', 'ACY1', 'AGS1', 'AGS2', 'AKC1', 'ATL2', 'AUS2', 'AUS3',
       'BCN1', 'BCN4', 'BDL2', 'BDL3', 'BDL4', 'BFI1', 'BFI4', 'BFL1',
       'BGY1', 'BHM1', 'BLQ1', 'BOI2', 'BRE4', 'BRQ2', 'BRS1', 'BRS2',
       'BWI2', 'BWU2', 'CLE2', 'CLE3', 'CLT4', 'CMH1', 'CMH4', 'DAL3',
       'DCA1', 'DEN3', 'DEN4', 'DET3', 'DET6', 'DFW7', 'DSA6', 'DSM5',
       'DTW1', 'DUS4', 'ELP1', 'EMA1', 'EMA2', 'ETZ2', 'EWR4', 'EWR9',
       'FAT1', 'FCO1', 'FRA7', 'FSD1', 'FTW6', 'FWA6', 'GEG1', 'GRR1',
       'GYR1', 'HAM2', 'HND6', 'HOU2', 'HOU6', 'IAH1', 'IGQ1', 'JAX2',
       'JFK8', 'KIX3', 'KIX5', 'KIX6', 'KTW3', 'LAS7', 'LCY2', 'LCY3',
       'LEJ5', 'LGA9', 'LGB3', 'LGB7', 'LIT1', 'LTN4', 'MAD7', 'MAN1',
       'MAN2', 'MAN3', 'MCO1', 'MDW7', 'MEM4', 'MIA1', 'MKC6', 'MKE1',
       'MKE2', 'MME1', 'MME2', 'MQY1', 'MSP1', 'MTN1', 'MUC3', 'MXP6',
       'NCL1', 'NCL2', 'NUE1', 'OAK4', 'OKC1', 'ORD5', 'ORF3', 'ORY4',
       'OVD1', 'OXR1', 'PAD1', 'PCW1', 'PDX9', 'POZ2', 'PSP1', 'PSR2',
      

In [23]:
# Checking Null % within each Column of the Export Dataframe

# Expanding # of Rows to be shown
pd.set_option('display.max_rows', 500)

print("Null Percentages by Column")
main_df.isnull().sum(axis = 0)/len(main_df) *100 

Null Percentages by Column


site                    0.000000
pt_fc_date              0.000000
total_stows             8.452682
hc_agreements           8.452682
ML_Accuracy            10.128952
hc_audits               8.452682
hc_count                8.452682
lc_agreements           8.452682
lc_audits               8.452682
lc_count                8.452682
multiple_events         8.452682
tron_bypass_rate        8.452682
tron_ioa                8.452682
me_dpmo                 8.452682
ml_accuracy             8.452682
ids_accuracy            8.452682
me_pct                  8.452682
ib_volume               7.279070
stow_hrs                7.279070
vet_hrs                 7.279070
stow_work_secs          7.279070
stows_performed         7.279070
IDS_work_secs           7.279070
IDS_stows_performed     7.279070
pod_visits              7.281299
units_per_face          7.281299
tenure_per              7.279070
ids_takt                8.756952
stow_gap_ms             4.785841
stow_total_ms           4.785841
stowgap   

In [24]:
# Exporting the Dataframe for Use in the Model Notebook

write_results_s3(main_df,'cotezach-prod','TRON_Bypass_Modelling/TBR_model_data_raw.csv')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/boto3/compat.py:88: PythonDeprecationWarning: Boto3 will no longer support Python 3.6 starting May 30, 2022. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.7 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
